In [496]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from imageio import imread
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook

In [44]:
bad_path = '../dataset/Bad50x50/27733/'
left_path = '../dataset/LeftHand50x50/27731/'
rigth_path = '../dataset/RightHand50x50/27730/'
test_path = '../dataset/Test189x110/27734/'

In [45]:
dir_path = [bad_path, left_path, rigth_path]

In [46]:
for path in dir_path:
    assert os.path.isdir(path)

In [462]:
X = []
y = []
for img_path in dir_path:
    for img_name in filter(lambda x: x.endswith('jpg'), os.listdir(img_path)):
        X.append(np.array(imread(os.path.join(img_path, img_name))[...,0]))
        y.append(img_path)
X = np.array(X)
y = np.array(y)

In [463]:
y = pd.get_dummies(y).values.argmax(axis=1)

In [464]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

In [465]:
X_train = torch.from_numpy(X_train/255)
X_test = torch.from_numpy(X_test/255)

In [466]:
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [467]:
X_train = X_train.view(-1, 1, 1, 50, 50)
X_test = X_test.view(-1, 1, 1, 50, 50)

In [470]:
y_train = y_train.view(-1, 1).long()
y_test = y_test.view(-1, 1).long()

In [526]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 9 * 9, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 9 * 9)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [544]:
net = Net()
net = net.float()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, (a, b) in  tqdm_notebook(enumerate(zip(X_train, y_train))):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = a, b

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if not i % 300 :    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.6f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
        correct = 0
        total = 0
    with torch.no_grad():
        for a, b in zip(X_test, y_test):
            images, labels = a, b
            outputs = net(images.float())
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += predicted.item() == labels.item()

    print(f'Accuracy of the network : {np.round(100 * correct / total, 3)}')
print('Finished Training')

[1,     1] loss: 0.000613
[1,   301] loss: 0.163890
[1,   601] loss: 0.162136
[1,   901] loss: 0.117527
[1,  1201] loss: 0.085416
[1,  1501] loss: 0.062879

Accuracy of the network : 85.842


[2,     1] loss: 0.000001
[2,   301] loss: 0.041245
[2,   601] loss: 0.036596
[2,   901] loss: 0.032533
[2,  1201] loss: 0.023478
[2,  1501] loss: 0.016045

Accuracy of the network : 96.416


[3,     1] loss: 0.000000
[3,   301] loss: 0.015476
[3,   601] loss: 0.011888
[3,   901] loss: 0.006257
[3,  1201] loss: 0.004700
[3,  1501] loss: 0.007364

Accuracy of the network : 99.642


[4,     1] loss: 0.000000
[4,   301] loss: 0.001829
[4,   601] loss: 0.001792
[4,   901] loss: 0.004252
[4,  1201] loss: 0.001811
[4,  1501] loss: 0.001748

Accuracy of the network : 99.821


[5,     1] loss: 0.000000
[5,   301] loss: 0.007470
[5,   601] loss: 0.000789
[5,   901] loss: 0.000327
[5,  1201] loss: 0.003571
[5,  1501] loss: 0.000215

Accuracy of the network : 99.642


[6,     1] loss: 0.000000
[6,   301] loss: 0.000173
[6,   601] loss: 0.000120
[6,   901] loss: 0.000113
[6,  1201] loss: 0.000065
[6,  1501] loss: 0.000057

Accuracy of the network : 99.821


[7,     1] loss: 0.000000
[7,   301] loss: 0.000054
[7,   601] loss: 0.000037
[7,   901] loss: 0.000022
[7,  1201] loss: 0.000041
[7,  1501] loss: 0.000036

Accuracy of the network : 99.821


[8,     1] loss: 0.000000
[8,   301] loss: 0.000024
[8,   601] loss: 0.000025
[8,   901] loss: 0.000018
[8,  1201] loss: 0.000027
[8,  1501] loss: 0.000026

Accuracy of the network : 99.821


[9,     1] loss: 0.000000
[9,   301] loss: 0.000019
[9,   601] loss: 0.000018
[9,   901] loss: 0.000014
[9,  1201] loss: 0.000020
[9,  1501] loss: 0.000020

Accuracy of the network : 99.821


[10,     1] loss: 0.000000
[10,   301] loss: 0.000015
[10,   601] loss: 0.000014
[10,   901] loss: 0.000011
[10,  1201] loss: 0.000016
[10,  1501] loss: 0.000016

Accuracy of the network : 99.821
Finished Training
